In [1]:
import pandas as pd
import numpy as np
import pyspark

# Establish Spark Context
`local[*]` lets pyspark know that we're using only one machine, as opposed to a set of machines in a cluster, etc.

The * tells `pyspark` to use all the cores on your machine.

It's important to not run more than once because once it's running, you'd have to kill the old SparkContext and start again. 

In [3]:
sc = pyspark.SparkContext('local[*]')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/04 16:28:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark = pyspark.sql.SparkSession(sc)

In [6]:
df = spark.read.csv('../data/NFIP/nfip-flood-policies.csv')

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string, _c12: string, _c13: string, _c14: string, _c15: string, _c16: string, _c17: string, _c18: string, _c19: string, _c20: string, _c21: string, _c22: string, _c23: string, _c24: string, _c25: string, _c26: string, _c27: string, _c28: string, _c29: string, _c30: string, _c31: string, _c32: string, _c33: string, _c34: string, _c35: string, _c36: string, _c37: string, _c38: string, _c39: string, _c40: string, _c41: string, _c42: string, _c43: string, _c44: string]

In [9]:
df.head()

23/04/04 16:29:25 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Row(_c0='agriculturestructureindicator', _c1='basefloodelevation', _c2='basementenclosurecrawlspacetype', _c3='cancellationdateoffloodpolicy', _c4='censustract', _c5='condominiumindicator', _c6='construction', _c7='countycode', _c8='crsdiscount', _c9='deductibleamountinbuildingcoverage', _c10='deductibleamountincontentscoverage', _c11='elevatedbuildingindicator', _c12='elevationcertificateindicator', _c13='elevationdifference', _c14='federalpolicyfee', _c15='floodzone', _c16='hfiaasurcharge', _c17='houseofworshipindicator', _c18='latitude', _c19='locationofcontents', _c20='longitude', _c21='lowestadjacentgrade', _c22='lowestfloorelevation', _c23='nonprofitindicator', _c24='numberoffloorsininsuredbuilding', _c25='obstructiontype', _c26='occupancytype', _c27='originalconstructiondate', _c28='originalnbdate', _c29='policycost', _c30='policycount', _c31='policyeffectivedate', _c32='policyterminationdate', _c33='policytermindicator', _c34='postfirmconstructionindicator', _c35='primaryreside